# ML Challenge Prediction Notebook

In [1]:
%matplotlib inline
%run ./../../plugins/widgets.py

import cv2
import glob
import ipywidgets
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import sklearn.model_selection
import time
import tensorflow as tf
import tensorflow.keras as keras

Widget Loaded


## Global parameters and variables

In [2]:
tf.enable_eager_execution()
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [3]:
IMG_SIZE = 224
IMG_SHAPE = [IMG_SIZE, IMG_SIZE, 3]

NUM_CLASSES = 102
BATCH_SIZE = 32

## Loading Filenames and Labels

In [4]:
df = pd.read_csv("./data/train.csv")
df['filename'] = './data/train/'+ df['image_id'].astype(str) + '.jpg'
df.head(5)

,image_id,category,filename
0,0,77,./data/train/0.jpg
1,1,81,./data/train/1.jpg
2,2,52,./data/train/2.jpg
3,3,72,./data/train/3.jpg
4,4,58,./data/train/4.jpg


In [5]:
filenames = []
labels = []
for index, row in df.iterrows():
    filenames.append(row['filename'])
    labels.append(row['category']-1)
    
print("Sample filenames : ", filenames[:4])
print("Sample labels : ", labels[:4])

Sample filenames :  ['./data/train/0.jpg', './data/train/1.jpg', './data/train/2.jpg', './data/train/3.jpg']
Sample labels :  [76, 80, 51, 71]


In [6]:
train_x, val_x, train_y, val_y = sklearn.model_selection.train_test_split(filenames, labels, test_size=0.20, random_state=64)
print("{} examples in training set".format(len(train_y)))
print("{} examples in validation set".format(len(val_y)))

14832 examples in training set
3708 examples in validation set


## Defining a DataGenerator

In [7]:
class DataGenerator(keras.utils.Sequence):
    
    # Generates data for Keras
    def __init__(self, filenames, labels, batch_size=BATCH_SIZE, n_classes=NUM_CLASSES, shuffle=True):   
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    # Generate one batch of data
    def __getitem__(self, batch_index):
        
        # Generate indexes of the batch
        batch_start = batch_index*self.batch_size
        batch_end = (batch_index+1)*self.batch_size
        
        batch_indices = self.indexes[batch_start:batch_end]
        
        batch_filenames = [self.filenames[i] for i in batch_indices]
        batch_labels = [self.labels[i] for i in batch_indices]

        batch_x = []
        batch_y = []
        
        # Generate data
        for i in range(self.batch_size):
            pil_image = keras.preprocessing.image.load_img(batch_filenames[i], target_size=(IMG_SIZE, IMG_SIZE))
            image_array = keras.preprocessing.image.img_to_array(pil_image)
            batch_x.append(image_array)
            batch_y.append(batch_labels[i])
        
        batch_x = np.array(batch_x)
        batch_y = keras.utils.to_categorical(batch_y, num_classes=self.n_classes)
        
        return batch_x, batch_y

    # Updates indexes after each epoch
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
          
    # Denotes the number of batches per epoch
    def __len__(self):
        return int(np.floor(len(self.filenames) / self.batch_size))

In [8]:
training_generator = DataGenerator(train_x, train_y)
validation_generator  = DataGenerator(val_x, val_y)

## Load and Tweak Pre-trained Model

In [9]:
# Loads the VGG19 Model without the last 2 FC layers when include_top=False
base_model = keras.applications.VGG19(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [10]:
x = base_model.output
x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=predictions)

In [11]:
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Train Model

In [13]:
%time
model.fit_generator(generator=training_generator, 
                    validation_data=validation_generator,
                    epochs=10)

W0811 22:56:38.087845  3544 deprecation.py:323] From C:\Program Files\Python 3.6.3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Wall time: 0 ns
Epoch 1/10
463/463 [==============================] - 167s 361ms/step - loss: 15.3411 - acc: 0.0471 - val_loss: 15.3441 - val_acc: 0.0470
Epoch 2/10
463/463 [==============================] - 163s 352ms/step - loss: 15.2626 - acc: 0.0528 - val_loss: 15.2605 - val_acc: 0.0527
Epoch 3/10
463/463 [==============================] - 161s 348ms/step - loss: 15.2044 - acc: 0.0559 - val_loss: 15.1109 - val_acc: 0.0625
Epoch 4/10
463/463 [==============================] - 164s 353ms/step - loss: 15.0508 - acc: 0.0657 - val_loss: 15.0863 - val_acc: 0.0633
Epoch 5/10
463/463 [==============================] - 165s 356ms/step - loss: 15.0163 - acc: 0.0682 - val_loss: 15.1335 - val_acc: 0.0609
Epoch 6/10
462/463 [============================>.] - ETA: 0s - loss: 14.9761 - acc: 0.0707

KeyboardInterrupt: 

## Make Predictions on Test Dataset

In [ ]:
# Make Predictions on test data
df = pd.read_csv("./data/test.csv")
df['filename'] = './data/test/' + df['image_id'].astype(str) + '.jpg'
df.head(5)

In [ ]:
pred_y = []
batch_x = []
for index, row in df.iterrows():
    pil_image = keras.preprocessing.image.load_img(row['filename'], target_size=(IMG_SIZE, IMG_SIZE))
    image_array = keras.preprocessing.image.img_to_array(pil_image)
    batch_x.append(image_array)

    if len(batch_x) >= BATCH_SIZE or index == len(df) - 1:
        batch_y = model.predict_on_batch(np.array(batch_x))
        batch_y = np.argmax(batch_y, axis=1) + 1
        pred_y.extend(batch_y)
        batch_x = []

df['category'] = pred_y
df.head(5)

In [ ]:
df[['image_id', 'category']].to_csv("./data/prediction.csv", index=False)